In [35]:
from lxml import html
import requests
import csv

page = requests.get('https://en.wiktionary.org/wiki/Appendix:1000_Japanese_basic_words')
doc  = html.fromstring(page.content)

headlines = doc.xpath('//span[@class="mw-headline"]')

def word_cleaner (word):
    word = word.replace(")", "")
    word = word.replace("ō","oo").replace("ī","ii").replace("ā","aa").replace("ū","uu").replace("ē", "ee")
    word = word.strip()
    return word

for part in headlines:
    tag = str(part.getparent())
    if"h3" in tag or "h4" in tag:
        if("Human made objects" in part.getparent().text_content()):
            continue

        filename = (part.getparent().text_content())
        with open("csvs\\" + filename + ".csv", 'w', newline='') as file:
            writer = csv.writer(file, delimiter =';', quoting=csv.QUOTE_NONE, escapechar='\\')
            for line in (part.getparent().getnext()):
                text = line.text_content()

                if("# -gatsu" in text or "Last year: kyonen," in text or "Yesterday morning:" in text or "(Last week: senshū, t" in text):
                    continue

                if("–" in text):
                    text = text.split("–")
                else:
                    text = text.split("-")

                if(len(text) <= 1):
                    print(text)

                text = text[1]
                text = text.split("(")

                if("te form of verb or adjective" in text[0]):
                    continue


                if(len(text) <= 1):
                    print(text)

                word = text[1]
                meaning = text[0]
                word = word_cleaner(word)
                meaning = word_cleaner(meaning)
                try:
                    writer.writerow([word, meaning])
                except:
                    print(word,meaning)

bunboogu stationery ​
e [Note: へ is normally read he, but when used as this particle its reading changes to e] to
tsutsu [similar to ながら but has different tone, and both actions have equal weight] ongoing occurrence


# csv to json

In [36]:
from os import listdir
from os.path import isfile, join
import csv, json

mypath = "csvs"
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

for file in onlyfiles:
    file = file.split(".")[0]
    csvPath = "csvs\\" + file + ".csv"

    data = {"wordPackName": file}

    words = []

    with open(csvPath) as csvFile:
        reader = csv.reader(csvFile, delimiter =';', quoting=csv.QUOTE_NONE, escapechar='\\')
        for rows in reader:
            words.append({"word" : rows[0], "meaning" : rows[1]})
            #print(rows)

    data["wordPairs"] = words
    data["$type"] = "WordPack"

    jsonPath = "jsons\\" + file + ".json"
    
    with open(jsonPath, "w") as jsonFile:
        jsonFile.write(json.dumps(data,indent=4))